In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 212.4MB 58kB/s 
     |████████████████████████████████| 204kB 16.8MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=2d368f2b7396b6d87ff13ec6f49d31ff099748e73712078783ed4be9db5b63f5
  Stored in directory: /root/.cache/pip/wheels/40/1b/2c/30f43be2627857ab80062bef1527c0128f7b4070b6b2d02139
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 39 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading

In [2]:
# Let's import the libraries we will need
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
import itertools

In [3]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")
# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [4]:
spark

In [5]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2021-06-14 17:47:52--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.205.198.58, 54.236.206.131, 34.235.3.193, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.205.198.58|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  6.05MB/s    in 2.2s    

2021-06-14 17:47:55 (6.05 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Traceback (most recent call last):
  File "<string>", line 1, in <module>
IndexError: list index out of range


In [ ]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
#gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 #print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))


In [21]:

supC = sc.broadcast(15061*0.22)

In [22]:
data = sc.textFile("Dataset.csv")

In [23]:
tagsheader = data.first() 
tags = sc.parallelize(tagsheader)
tags.collect()
data = data.subtract(tags)

In [24]:
data1 = data.map(lambda x: [i.strip().replace('.','') for i in x.split(',') if i not in ''])

In [25]:
counts = data1.flatMap(lambda x: [(i, 1) for i in x]).reduceByKey(lambda x, y: x+y)

In [50]:
freq = counts.filter(lambda x: x[1]>supC.value).map(lambda x: x[0])

In [51]:
from pyspark.sql.functions import udf
from pyspark.sql.types import BooleanType

def isPresent(combination, u, i):
      
    if i<2:
        return combination in u
    subs = [tuple(sorted(combination)) for combination in itertools.combinations(combination, i-1)]
    # return subs.issubset(prevFreq)
    present = [False]*len(subs)
    for j in range(len(subs)):
        present[j]= subs[j] in u
    # print('present at iter', i , subs ,present)
    return all(present)
    

def f(row, arr):
    hazir= []
    for key in arr:
        present = [False]*len(key)
        for i in range(len(key)):
            present[i] = key[i] in row
        if all(present):
            hazir+=[key]
    return hazir
    

In [52]:
home = '/content/'
for i in range(2, 11):
    
    u = freq.collect()
    print('frequent at iteration', i-1, u)
    print('===============sequence length=============' ,i)
    with open('output'+str(i)+'.txt', 'w+') as file:
      for item in u:
          if type(item) is tuple:
              length=len(item)
              file.write("(")
              for k in range(0,length-1):
                  file.write(str(item[k])+",")
              file.write(str(item[length-1])+")\n")
          else:
              file.write(str(item) + "\n")
    file.close()
    if i==2:
        distinct = u
        combos = [tuple(sorted(combination)) for combination in itertools.combinations(distinct, i)]
        # print('distinct[0]\n',distinct)
    else:
        # print('freq at iteration',i,'\n',u)
        distinct = set([])
        for y in u:
            for x in y:
                distinct.add(x)
        # print('distinct[0]\n',distinct)
        combos = [combination for combination in list(itertools.combinations(list(distinct), i)) if isPresent(combination, u, i)]
    # print('combos[0]\n', combos)
    
    
    comb = sc.broadcast(combos)
    freq1 = data1.map(lambda x: [(key, 1) for key in f(x, comb.value)]).filter(lambda x: len(x)>0)
    # print('freq1', freq1.collect())

    freq2 = freq1.flatMap(lambda x: x).reduceByKey(lambda x, y: x+y).filter(lambda x: x[1]>supC.value)
    # print('freq2', freq2.collect())
    if freq2.count() == 0:
        print('ending at sequence length' ,i-1)
        break
    freq = freq2.map(lambda x: x[0])

    
    
freq.collect()

frequent at iteration 1 ['Husband', 'White', 'United-States', 'Normal-Hours', 'Not-in-family', 'Married', 'Lowwer-Working-Class', 'Technical-Working-Class', 'Male', 'Part-Time-Hours', '>50K', 'Private', '9', 'Middle-Age', 'Female', 'Young-Age', 'HS-Graduates', '<=50K', 'Other-Service', 'Never-married']
===============sequence length============= 2
frequent at iteration 2 [('Husband', 'Male'), ('Husband', 'Part-Time-Hours'), ('Male', 'White'), ('Part-Time-Hours', 'White'), ('>50K', 'White'), ('Male', 'United-States'), ('Part-Time-Hours', 'United-States'), ('>50K', 'United-States'), ('Husband', 'Private'), ('Private', 'White'), ('9', 'White'), ('Private', 'United-States'), ('9', 'United-States'), ('Middle-Age', 'White'), ('Middle-Age', 'United-States'), ('<=50K', 'HS-Graduates'), ('Female', 'United-States'), ('Female', 'White'), ('White', 'Young-Age'), ('United-States', 'Young-Age'), ('HS-Graduates', 'Other-Service'), ('<=50K', 'Never-married'), ('<=50K', 'Other-Service'), ('Husband', 'M

[('Male', 'Part-Time-Hours', 'Married'),
 ('United-States', 'Husband', 'Married'),
 ('United-States', 'White', 'Married'),
 ('Husband', 'White', 'Married'),
 ('Male', 'HS-Graduates', '9'),
 ('Male', 'HS-Graduates', 'Part-Time-Hours'),
 ('Male', 'HS-Graduates', 'Private'),
 ('Male', 'Private', 'Married'),
 ('HS-Graduates', 'United-States', '9'),
 ('HS-Graduates', 'United-States', 'Part-Time-Hours'),
 ('HS-Graduates', 'United-States', 'Private'),
 ('HS-Graduates', '9', 'White'),
 ('HS-Graduates', 'Part-Time-Hours', 'White'),
 ('HS-Graduates', 'White', 'Private'),
 ('Male', 'United-States', '<=50K'),
 ('Male', 'Husband', '<=50K'),
 ('Male', 'White', '<=50K'),
 ('Male', '<=50K', 'Private'),
 ('United-States', '9', '<=50K'),
 ('United-States', 'Part-Time-Hours', '<=50K'),
 ('9', 'White', '<=50K'),
 ('Part-Time-Hours', 'White', '<=50K'),
 ('Part-Time-Hours', '<=50K', 'Private'),
 ('United-States', 'Other-Service', 'White'),
 ('United-States', 'Female', '<=50K'),
 ('Female', '<=50K', 'Private